In [1]:
# Toolboxes and Data Import
import scipy.io as spio
import numpy as np
import pandas as pd

mat = spio.loadmat('WorstOffAutocallablePayoff_inputs.mat', squeeze_me=True)

In [13]:
!dir


 Volume in drive C is OS
 Volume Serial Number is D8EE-33B1

 Directory of C:\Users\davide\Desktop\Intesa-prod-Governance

07/22/2019  05:46 PM    <DIR>          .
07/22/2019  05:46 PM    <DIR>          ..
07/19/2019  03:18 PM    <DIR>          .ipynb_checkpoints
07/18/2019  02:41 PM             4,800 CouponFixed.csv
04/01/2019  01:56 PM         6,345,873 FMR4000FunctionReference.pdf
07/22/2019  11:56 AM           303,002 Payoff prova.ipynb
07/19/2019  10:16 AM            26,566 Payoff.csv
07/08/2019  01:54 PM    <DIR>          Payoffs_ProductGovernance_ISP
07/18/2019  02:41 PM             8,114 Payoff_2.csv
07/19/2019  09:51 AM             8,705 Payoff_3.csv
07/19/2019  10:16 AM            26,566 Payoff_4.csv
12/20/2018  03:07 PM        12,243,676 PV - Instrument Configuration.xlsx
07/22/2019  03:16 PM            40,941 WorstOffAutoCallablePayoff.ipynb
07/22/2019  05:46 PM            28,686 WorstOffAutocallablePayoffPy.ipynb
07/17/2019  04:14 PM            39,949 WorstOffAutocallableP

In [2]:
# Payoff function definition

def worst_off_autocallable_payoff(para_prod, risk_factor, barrier_factor,num_coupon_fix,num_sim):
    
    # Index Calculation
    rf1 = np.divide(risk_factor[:,:,1:],risk_factor[:,:,[0]])
    num_event = rf1.shape[2]
    
    # Correction for Worst of
    rf = np.amin(rf1,axis=1)
    bf1 = barrier_factor / risk_factor[:,:,0]
    wo1 = np.min(bf1, axis=1)
    bf = wo1
        
    # Digital Effect
    pointer_digital_effect = para_prod[-(num_coupon_fix*3):-(num_coupon_fix*2), None].T        
    coupon_digital = para_prod[(num_coupon_fix*2):(num_coupon_fix*3)]
    digital_effect = (np.greater_equal(rf, para_prod[:num_coupon_fix]) * coupon_digital ) * pointer_digital_effect
    
    # Autocall Effect
    pointer_autocall_effect = para_prod[-(num_coupon_fix*2):-(num_coupon_fix), None].T
    
    # Pointer Matrix
    pointer_auto_call = np.greater_equal(rf, para_prod[num_coupon_fix:num_coupon_fix*2]).astype(int)
    pointer_auto_call[:, np.where(pointer_autocall_effect == 0)] = 0
    pointer_auto_call = np.cumsum(pointer_auto_call, axis=1)
    pointer_auto_call[np.where(pointer_auto_call > 1)] = 30000
    pointer_auto_call[np.where(pointer_auto_call[:,-1] == 0)] = 10000
    pointer_auto_call[np.where(pointer_auto_call == 10000)] = 3
    pointer_auto_call[np.where(pointer_auto_call == 20000)] = 2
    pointer_auto_call[np.where(pointer_auto_call == 30000)] = 0
    
    # Autocall Coupon T-
    auto_call_coupon = digital_effect
    auto_call_coupon[np.where(pointer_auto_call == 1)] += 1
    auto_call_coupon[np.where(pointer_auto_call == 0)] = 0
    
    # Autocall Coupon T
    pointer_auto_call_end = pointer_auto_call[:,-1]
    payoff_end = np.copy(bf)
    payoff_end[np.greater_equal(bf,para_prod[num_coupon_fix*2-1])] = para_prod[num_coupon_fix*3]
    payoff_end[np.greater_equal(bf,para_prod[num_coupon_fix-1])] = para_prod[num_coupon_fix*3+1]
    payoff_end[np.where(pointer_auto_call_end != 3)] = 0
    auto_call_coupon[:,-1] = payoff_end
    
    # Fix Coupon
    coupon_fix = np.repeat(para_prod[-(num_coupon_fix):][np.newaxis].T, num_sim, axis=1).T
    payoff = auto_call_coupon
    
    return payoff, coupon_fix

In [3]:
# Function Execution
Payoff, CouponFix = worst_off_autocallable_payoff(mat['ParaProd'], mat["RiskFactor"], mat["BarrierFactor"], mat["NumCouponFix"], mat["NumSim"])

In [4]:
# Results
Payoff = pd.DataFrame(Payoff)
CouponFix = pd.DataFrame(CouponFix)

In [5]:
# Load Matlab Results
MatPayoff = pd.read_csv('Payoff.csv', header=None)
MatCouponFix = pd.read_csv('CouponFixed.csv', header=None)

In [6]:
# Payoff Differences
payoff_diff = ((Payoff - MatPayoff) / MatPayoff).round(15)
payoff_diff.fillna(0, inplace=True)

# CouponFix Differences
coupon_fix_diff = ((CouponFix - MatCouponFix) / MatCouponFix).round(15)
coupon_fix_diff.fillna(0, inplace=True)

In [7]:
# Payoff Results
pd.DataFrame(np.transpose([Payoff.iloc[:,-1], MatPayoff.iloc[:,-1], payoff_diff.iloc[:,-1].abs()]), columns=["Payoff Python", "Payoff Matlab", "Differences"])

,Payoff Python,Payoff Matlab,Differences
0,0.033646,0.033646,0.0
1,0.075774,0.075774,0.0
2,0.201629,0.201629,0.0
3,0.110093,0.110093,0.0
4,1.000000,1.000000,0.0
5,1.010000,1.010000,0.0
6,1.010000,1.010000,0.0
7,0.231149,0.231149,0.0
8,0.095644,0.095644,0.0
9,1.010000,1.010000,0.0


In [8]:
# Payoff Sum Differences
payoff_diff.sum().sum()

0.0

In [9]:
# Payoff differences Min and Max
pd.DataFrame([payoff_diff.min(), payoff_diff.max()])

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Results CouponFix Sum
coupon_fix_diff.sum().sum()

0.0

In [11]:
# CouponFix differences Min and Max
pd.DataFrame([coupon_fix_diff.min(), coupon_fix_diff.max()])

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
